# 10. Advanced Expression Manipulation
http://docs.sympy.org/latest/tutorial/manipulation.html

In [1]:
from sympy import *
init_session()

IPython console for SymPy 1.0 (Python 3.6.0-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at http://docs.sympy.org/1.0/


In this section, we discuss some ways that we can perform advanced manipulation of expressions.

## Understanding Expression Trees
Before we can do this, we need to understand how expressions are represented in SymPy. A mathematical expression is represented as a tree. Let us take the expression $x^2 + xy$, i.e., $x**2 + x*y$. We can see what this expression looks like internally by using ***srepr***

In [2]:
expr = x**2 + x*y
expr

In [3]:
srepr(expr)

"Add(Pow(Symbol('x'), Integer(2)), Mul(Symbol('x'), Symbol('y')))"

The easiest way to tear this apart is to look at a diagram of the expression tree:
<img src='http://docs.sympy.org/latest/_images/graphviz-d8d6268439273f5c9659310fcb25b50fd18c6260.svg'>

First, let’s look at the leaves of this tree. Symbols are instances of the class ***Symbol***. While we have been doing

In [4]:
x = symbols('x')
x

we could have also done

In [6]:
x = Symbol('x')
x

Either way, we get a Symbol with the name “x” [1]. For the number in the expression, 2, we got ***Integer(2)***. Integer is the SymPy class for integers. It is similar to the Python built-in type int, except that Integer plays nicely with other SymPy types.

When we write $x**2$, this creates a ***Pow*** object. Pow is short for “power”.

In [7]:
srepr(x**2)

"Pow(Symbol('x'), Integer(2))"

We could have created the same object by calling Pow(x, 2)

In [8]:
Pow(x, 2)

Note that in the srepr output, we see Integer(2), the SymPy version of integers, even though technically, we input 2, a Python int. In general, whenever you combine a SymPy object with a non-SymPy object via some function or operation, the non-SymPy object will be converted into a SymPy object. The function that does this is ***sympify*** [2].

In [9]:
type(2)

int

In [10]:
type(simplify(2))

sympy.core.numbers.Integer

We have seen that $x**2$ is represented as Pow(x, 2). What about $x*y$? As we might expect, this is the multiplication of x and y. The SymPy class for multiplication is ***Mul***.

In [12]:
srepr(x*y)

"Mul(Symbol('x'), Symbol('y'))"

Thus, we could have created the same object by writing Mul(x, y).

In [13]:
Mul(x, y)

Now we get to our final expression, $x**2 + x*y$. This is the addition of our last two objects, Pow(x, 2), and Mul(x, y). The SymPy class for addition is ***Add***, so, as you might expect, to create this object, we use Add(Pow(x, 2), Mul(x, y)).

In [14]:
Add(Pow(x, 2), Mul(x, y))

SymPy expression trees can have many branches, and can be quite deep or quite broad. Here is a more complicated example

In [15]:
expr = sin(x*y)/2 - x**2 + 1/y
expr

In [16]:
srepr(expr)

"Add(Mul(Integer(-1), Pow(Symbol('x'), Integer(2))), Mul(Rational(1, 2), sin(Mul(Symbol('x'), Symbol('y')))), Pow(Symbol('y'), Integer(-1)))"

Here is a diagram
<img src='http://docs.sympy.org/latest/_images/graphviz-a15fbf4039d3f5488f10cf31b136816de047baa7.svg'>

This expression reveals some interesting things about SymPy expression trees. Let’s go through them one by one.

Let’s first look at the term $x**2$. As we expected, we see Pow(x, 2). One level up, we see we have Mul(-1, Pow(x, 2)). ***There is no subtraction class in SymPy***. x - y is represented as x + -y, or, more completely, x + -1*y, i.e., Add(x, Mul(-1, y)).

In [17]:
expr = x - y
srepr(expr)

"Add(Symbol('x'), Mul(Integer(-1), Symbol('y')))"

<img src='http://docs.sympy.org/latest/_images/graphviz-751d8866c8b62cfbeb5393714bd4f17241679175.svg'>

Next, look at 1/y. We might expect to see something like Div(1, y), but similar to subtraction, ***there is no class in SymPy for division***. Rather, division is represented by a ***power of -1***. Hence, we have Pow(y, -1). What if we had divided something other than 1 by y, like x/y? Let’s see.

In [18]:
srepr(x / y)

"Mul(Symbol('x'), Pow(Symbol('y'), Integer(-1)))"

<img src='http://docs.sympy.org/latest/_images/graphviz-e878bf0458393c212427a889ec15220a360a91b4.svg'>

We see that x/y is represented as $x*y**-1, i.e., Mul(x, Pow(y, -1))$.

Finally, let’s look at the $sin(x*y)/2$ term. Following the pattern of the previous example, we might expect to see $Mul(sin(x*y), Pow(Integer(2), -1))$. But instead, we have $Mul(Rational(1, 2), sin(x*y))$. ***Rational numbers are always combined into a single term in a multiplication***, so that when we divide by 2, it is represented as multiplying by 1/2.

Finally, one last note. You may have noticed that the ***order*** we entered our expression and the order that it came out from srepr or in the graph were different. You may have also noticed this phenonemon earlier in the tutorial. For example

In [19]:
1 + x

This because in SymPy, the arguments of the ***commutative operations Add and Mul are stored in an arbitrary (but consistent!) order***, which is independent of the order inputted (if you’re worried about noncommutative multiplication, don’t be. In SymPy, you can create noncommutative Symbols using Symbol('A', ***commutative=False***), and the order of multiplication for noncommutative Symbols is kept the same as the input). Furthermore, as we shall see in the next section, the printing order and the order in which things are stored internally need not be the same either.

#### Quick Tip
The way an expression is represented internally and the way it is printed are often not the same.

In general, an important thing to keep in mind when working with SymPy expression trees is this: ***the internal representation of an expression and the way it is printed need not be the same***. The same is true for the input form. If some expression manipulation algorithm is not working in the way you expected it to, chances are, the internal representation of the object is different from what you thought it was.

## Recursing through an Expression Tree
Now that you know how expression trees work in SymPy, let’s look at how to dig our way through an expression tree. Every object in SymPy has two very important attributes, ***func, and args***.

### func
func is the ***head of the object***. For example, (x*y).func is Mul. Usually it is the same as the class of the object (though there are exceptions to this rule).

Two notes about func:

- First, the class of an object need not be the same as the one used to create it. For example

In [21]:
expr = Add(x, x)
srepr(expr)

"Mul(Integer(2), Symbol('x'))"

In [22]:
expr.func

sympy.core.mul.Mul

In [24]:
expr.args

We created Add(x, x), so we might expect expr.func to be Add, but instead we got Mul. Why is that? Let’s take a closer look at expr.

In [23]:
expr

Add(x, x), i.e., x + x, was automatically converted into Mul(2, x), i.e., $2*x$, which is a Mul. SymPy classes make heavy use of the __new__ class constructor, which, unlike __init__, allows a different class to be returned from the constructor.

- Second, some classes are special-cased, usually for efficiency reasons [3].

In [25]:
Integer(2).func

sympy.core.numbers.Integer

In [30]:
Integer(1).func

sympy.core.numbers.One

In [26]:
Integer(0).func

sympy.core.numbers.Zero

In [27]:
Integer(-1).func

sympy.core.numbers.NegativeOne

In [28]:
Integer(-2).func

sympy.core.numbers.Integer

or the most part, these issues will not bother us. The special classes ***Zero, One, NegativeOne***, and so on are subclasses of Integer, so as long as you use isinstance, it will not be an issue.

### args
args are the top-level arguments of the object. (x*y).args would be (x, y). Let’s look at some examples

In [31]:
expr = 3 * y**2 * x
expr

In [33]:
srepr(expr)

"Mul(Integer(3), Symbol('x'), Pow(Symbol('y'), Integer(2)))"

In [32]:
expr.args

From this, we can see that $expr == Mul(3, y**2, x)$. In fact, we can see that we can completely reconstruct expr from its func and its args.

In [35]:
expr.func(*expr.args)

In [36]:
expr == expr.func(*expr.args)

True

Note that although we entered 3*y**2*x, the args are (3, x, y**2).   

In a Mul, the Rational coefficient will come first in the args, but other than that, the order of everything else follows no special pattern. To be sure, though, there is an order.

In [37]:
expr = y**2 * 3 * x
expr.args

Mul’s args are sorted, so that the same Mul will have the same args. But the sorting is based on some criteria designed to make the sorting unique and efficient that has no mathematical significance.

The srepr form of our expr is Mul(3, x, Pow(y, 2)). What if we want to get at the args of Pow(y, 2). Notice that the $y**2$ is in the third slot of expr.args, i.e., expr.args[2].

In [38]:
expr.args[2]

So to get the args of this, we call expr.args[2].args.

In [39]:
expr.args[2].args

Now what if we try to go deeper. What are the args of y. Or 2. Let’s see.

In [40]:
expr.args[2].args[0].args

In [41]:
y.args

In [42]:
Integer(2).args

They both have ***empty args***. In SymPy, empty args signal that we have hit a ***leaf*** of the expression tree.

So there are two possibilities for a SymPy expression. Either:

- it has empty args, in which case it is a leaf node in any expression tree, or
- it has args, in which case, it is a branch node of any expression tree. 

***When it has args, it can be completely rebuilt from its func and its args***. This is expressed in the key invariant.

#### Key Invariant
Every well-formed SymPy expression must either have ***empty args*** or satisfy ***expr == expr.func(*expr.args)***.  

(Recall that in Python if a is a tuple, then f(*a) means to call f with arguments from the elements of a, e.g., f(*(1, 2, 3)) is the same as f(1, 2, 3).)

This key invariant allows us to write simple algorithms that walk expression trees, change them, and rebuild them into new expressions.


## Walking the Tree
With this knowledge, let’s look at how we can recurse through an expression tree. The nested nature of args is a perfect fit for recursive functions. The base case will be empty args. Let’s write a simple function that goes through an expression and prints all the args at each level.

In [47]:
def pre(expr):
    print(expr)
    for arg in expr.args:
        pre(arg)

See how nice it is that () signals leaves in the expression tree. We don’t even have to write a base case for our recursion; it is handled automatically by the for loop.

Let’s test our function.

In [48]:
pre(x * y + 1)

x*y + 1
1
x*y
x
y


Can you guess why we called our function pre? We just wrote a ***pre-order traversal function*** for our expression tree. See if you can write a post-order traversal function.

Such traversals are so common in SymPy that the generator functions preorder_traversal and postorder_traversal are provided to make such traversals easy. We could have also written our algorithm as

In [49]:
for arg in preorder_traversal(expr):
    print(arg)

3*x*y**2
3
x
y**2
y
2


### Footnotes

1. We have been using symbols instead of Symbol because it automatically splits apart strings into multiple Symbols. symbols('x y z') returns a tuple of three Symbols. Symbol('x y z') returns a single Symbol called x y z.
2. Technically, it is an internal function called _sympify, which differs from sympify in that it does not convert strings. x + '2' is not allowed.
3. Classes like One and Zero are ***singletonized***, meaning that only one object is ever created, no matter how many times the class is called. This is done for ***space efficiency***, as these classes are very common. For example, Zero might occur very often in a sparse matrix represented densely. As we have seen, NegativeOne occurs any time we have -x or 1/x. It is also done for ***speed efficiency*** because singletonized objects ***can be compared by is***. The unique objects for each singletonized class can be accessed from the ***S*** object.